## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층

In [1]:
class MulLayer:
    def __init__(self): # 인스턴스 변수를 초기화한다.
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


### 5.4.2 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
sum_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num) #(1)
orange_price = mul_orange_layer.forward(orange, orange_num) #(2)
all_price = sum_apple_orange_layer.forward(apple_price, orange_price) #(3)
price = mul_tax_layer.forward(all_price, tax) #(4)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) #(4)
dapple_price, dorange_price = sum_apple_orange_layer.backward(dall_price) #(3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) #(2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) #(1)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 5.7 오차역전파법 구현하기

In [6]:
# coding: utf-8
import sys, os
sys.path.append("../week3/")
sys.path.append("../week3/ch05/")# 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

In [7]:
help(TwoLayerNet)

Help on class TwoLayerNet in module two_layer_net:

class TwoLayerNet(builtins.object)
 |  TwoLayerNet(input_size, hidden_size, output_size, weight_init_std=0.01)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  accuracy(self, x, t)
 |  
 |  gradient(self, x, t)
 |  
 |  loss(self, x, t)
 |      # x : 입력 데이터, t : 정답 레이블
 |  
 |  numerical_gradient(self, x, t)
 |      # x : 입력 데이터, t : 정답 레이블
 |  
 |  predict(self, x)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [8]:
import inspect

inspect.getsource(TwoLayerNet)

"class TwoLayerNet:\n\n    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):\n        # 가중치 초기화\n        self.params = {}\n        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)\n        self.params['b1'] = np.zeros(hidden_size)\n        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) \n        self.params['b2'] = np.zeros(output_size)\n\n        # 계층 생성\n        self.layers = OrderedDict()\n        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])\n        self.layers['Relu1'] = Relu()\n        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])\n\n        self.lastLayer = SoftmaxWithLoss()\n        \n    def predict(self, x):\n        for layer in self.layers.values():\n            x = layer.forward(x)\n        \n        return x\n        \n    # x : 입력 데이터, t : 정답 레이블\n    def loss(self, x, t):\n        y = self.predict(x)\n        return self.lastLay

### 5.7.2 오차역전파법을 적용한 신경망 구현하기

#### 오차역전파법으로 구한 기울기 검증하기

In [9]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch) # 수치미분
grad_backprop = network.gradient(x_batch, t_batch) # 역전파

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:9.54137763141503e-07
b1:1.2985355471212502e-05
W2:4.924179290177508e-09
b2:1.39495206730178e-07


In [12]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 15001
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    # grad = network.numerical_gradient(x_batch, t_batch) # 수치미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차 역전파법 방식(훨씬 빠름)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("i: {}  train_acc: {}  test_acc: {}  ".format(i, train_acc, test_acc))

i: 0  train_acc: 0.08665  test_acc: 0.0833  
i: 600  train_acc: 0.9036  test_acc: 0.9047  
i: 1200  train_acc: 0.92255  test_acc: 0.9259  
i: 1800  train_acc: 0.93595  test_acc: 0.9356  
i: 2400  train_acc: 0.9434666666666667  test_acc: 0.9414  
i: 3000  train_acc: 0.9512333333333334  test_acc: 0.9496  
i: 3600  train_acc: 0.9561333333333333  test_acc: 0.952  
i: 4200  train_acc: 0.9598166666666667  test_acc: 0.9563  
i: 4800  train_acc: 0.9633666666666667  test_acc: 0.9584  
i: 5400  train_acc: 0.9668833333333333  test_acc: 0.9632  
i: 6000  train_acc: 0.9696166666666667  test_acc: 0.9645  
i: 6600  train_acc: 0.9706666666666667  test_acc: 0.9653  
i: 7200  train_acc: 0.9729166666666667  test_acc: 0.9665  
i: 7800  train_acc: 0.9735333333333334  test_acc: 0.9661  
i: 8400  train_acc: 0.9752833333333333  test_acc: 0.9677  
i: 9000  train_acc: 0.9751333333333333  test_acc: 0.9684  
i: 9600  train_acc: 0.9769166666666667  test_acc: 0.9698  
i: 10200  train_acc: 0.9794166666666667  test_a

In [11]:
train_size

60000